pipei1

In [ ]:
from transformers import BertModel, BertTokenizer
import networkx as nx
import torch
import numpy as np

def load_bert_model():
    # 加载预训练的BERT模型和tokenizer
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    return model, tokenizer

def get_node_embeddings_bert(graph, bert_model, tokenizer):
    node_embeddings = {}
    for node in graph.nodes():
        # 对节点中的单词或短语进行BERT embedding
        inputs = tokenizer(node, return_tensors='pt')
        outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
        node_embeddings[node] = embeddings
    return node_embeddings

def cosine_similarity(embedding1, embedding2):
    # 计算两个节点的余弦相似度
    similarity = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    return similarity

def calculate_semantic_node_similarity(graph1, graph2, bert_model, tokenizer):
    node_embeddings1 = get_node_embeddings_bert(graph1, bert_model, tokenizer)
    node_embeddings2 = get_node_embeddings_bert(graph2, bert_model, tokenizer)

    # 计算节点相似性的平均值
    similarities = [cosine_similarity(node_embeddings1[node1], node_embeddings2[node2])
                    for node1 in node_embeddings1
                    for node2 in node_embeddings2]

    # 取相似性的平均值
    similarity = np.mean(similarities) if similarities else 0
    return similarity

# 加载预训练的BERT模型和tokenizer
bert_model, tokenizer = load_bert_model()

# 构建两个图谱
graph1 = nx.Graph()
graph1.add_nodes_from(['cat', 'dog', 'apple'])
graph1.add_edges_from([('cat', 'dog'), ('dog', 'apple')])

graph2 = nx.Graph()
graph2.add_nodes_from(['kitten', 'puppy', 'fruit'])
graph2.add_edges_from([('kitten', 'puppy'), ('puppy', 'fruit')])

# 计算语义节点相似性
semantic_similarity = calculate_semantic_node_similarity(graph1, graph2, bert_model, tokenizer)

print(f"语义节点相似性：{semantic_similarity}")


other


In [3]:
from data_utils import build_tokenizer, build_embedding_matrix, Tokenizer4Bert, ABSADataset
import pickle


In [4]:
tokenizer = pickle.load(open('/home/dingdaijun/data_list/dingdaijun/LTN/dat/dt_tokenizer.dat', 'rb'))

In [7]:
a = "{text} if best"

In [9]:
text = 'i am god'
a.format(text=text)

'i am god if best'

In [15]:
tokenizer.word2idx

{'(': 1,
 ')': 2,
 '¬': 3,
 '∨': 4,
 '∧': 5,
 '→': 6,
 'they': 1,
 'legalized': 2,
 'gay': 3,
 'marriage???!!!': 4,
 "what's": 5,
 'next?!?!?': 6,
 'crack?!!!?': 7,
 'marriage': 8,
 'x': 9,
 'is': 10,
 'the': 11,
 'net': 12,
 'thing': 13,
 'crack': 14,
 'neutral': 15,
 'donald': 16,
 'trump': 17,
 'attitude': 18,
 'towards': 19,
 'given': 20,
 'target': 21,
 '"donald': 22,
 'trump"': 23,
 'neutral@whiphoyer': 24,
 '@corlessjones': 25,
 'haha': 26,
 'aca': 27,
 'far': 28,
 'from': 29,
 'affordable': 30,
 'you': 31,
 'bunch': 32,
 'of': 33,
 'gruber': 34,
 'idiots!': 35,
 'not': 36,
 'a': 37,
 'idiot': 38,
 'opposed': 39,
 'oppose': 40,
 'trump"grate': 41,
 'two': 42,
 'see': 43,
 '@realdonaldtrump': 44,
 'get': 45,
 'tuff': 46,
 'on': 47,
 'boarder': 48,
 'control!': 49,
 'has': 50,
 'my': 51,
 'vote': 52,
 'in': 53,
 '2016!': 54,
 '#tcot': 55,
 '#illegalsstolejobs': 56,
 'tough': 57,
 'border': 58,
 'control': 59,
 'supported': 60,
 'support': 61,
 'trump"@realdonaldtrump': 62,
 'we': 

In [14]:
tokenizer.idx2word[2]

'legalized'

In [11]:
for item in [1, 37, 3, 262, 9, 5]:
    print(tokenizer.idx2word[item])

they
a
gay
person
x
what's


In [2]:
#评估函数
from sklearn import metrics
def cal_f1(y_pred, y_true, two_class=True):
    if two_class:
        labels = [0, 1]
    else:
        labels = [0, 1, 2]
    macro_f1_all = metrics.f1_score(y_true, y_pred, labels=[0, 1, 2], average='macro')
    macro_f1 = metrics.f1_score(y_true, y_pred, labels=[0, 1], average='macro')
    f_favor = metrics.f1_score(y_true, y_pred, labels=[0], average='macro')
    f_against = metrics.f1_score(y_true, y_pred, labels=[1], average='macro')
    f_none = metrics.f1_score(y_true, y_pred, labels=[2], average='macro')
    # m_f_2 = metrics.f1_score(y_true, y_pred, labels=[0,1], average='macro')
    # print(table)
    return {"macro_f1_all": round(macro_f1_all*100, 2),
            "macro_f": round(macro_f1*100, 2),
            "f_favor": round(f_favor*100, 2),
            "f_against": round(f_against*100, 2),
            "f_none": round(f_none*100, 2)}

In [4]:
import csv
preds = []
labels = []
mtm = {'NONE':2,'AGAINST':0,'FAVOR':1,'support':1,'neutral':2,'opposed':0}
with open('../datasets/sem16/dt/dt.csv') as f:
    ls = csv.DictReader(f)
    for l1 in ls:
        preds.append(mtm[l1['Pred']])
        labels.append(mtm[l1['Stance']])
cal_f1(preds,labels)


{'macro_f1_all': 72.32,
 'macro_f': 69.58,
 'f_favor': 62.37,
 'f_against': 76.79,
 'f_none': 77.8}

In [5]:
import csv
preds = []
labels = []
mtm = {'NONE':2,'AGAINST':0,'FAVOR':1,'support':1,'neutral':2,'opposed':0}
with open('../datasets/sem16/fm/fm.csv') as f:
    ls = csv.DictReader(f)
    for l1 in ls:
        preds.append(mtm[l1['Pred']])
        labels.append(mtm[l1['Stance']])
cal_f1(preds,labels)

{'macro_f1_all': 66.09,
 'macro_f': 70.3,
 'f_favor': 74.07,
 'f_against': 66.54,
 'f_none': 57.66}

In [6]:
import csv
preds = []
labels = []
mtm = {'NONE':2,'AGAINST':0,'FAVOR':1,'support':1,'neutral':2,'opposed':0}
with open('../datasets/sem16/la/la.csv') as f:
    ls = csv.DictReader(f)
    for l1 in ls:
        preds.append(mtm[l1['Pred']])
        labels.append(mtm[l1['Stance']])
cal_f1(preds,labels)

{'macro_f1_all': 67.96,
 'macro_f': 68.43,
 'f_favor': 73.51,
 'f_against': 63.34,
 'f_none': 67.02}

In [3]:
import csv
preds = []
labels = []
mtm = {'NONE':2,'AGAINST':0,'FAVOR':1,'support':1,'neutral':2,'opposed':0}
with open('../datasets/sem16/hc/hc.csv') as f:
    ls = csv.DictReader(f)
    for l1 in ls:
        preds.append(mtm[l1['Pred']])
        labels.append(mtm[l1['Stance']])
cal_f1(preds,labels)

{'macro_f1_all': 76.52,
 'macro_f': 77.98,
 'f_favor': 78.87,
 'f_against': 77.09,
 'f_none': 73.6}

In [5]:
import csv
preds = []
labels = []
mtm = {'NONE':2,'AGAINST':0,'FAVOR':1,'support':1,'neutral':2,'opposed':0,'1':1,'0':0,'2':2}
with open('../datasets/sem16/vast_10/test.csv') as f:
    ls = csv.DictReader(f)
    for l1 in ls:
        preds.append(mtm[l1['Pred']])
        labels.append(mtm[l1['Stance']])
cal_f1(preds,labels)

{'macro_f1_all': 79.81,
 'macro_f': 75.2,
 'f_favor': 77.65,
 'f_against': 72.75,
 'f_none': 89.04}

In [24]:
import csv
preds1 = []
labels2 = []
mtm = {'NONE':2,'AGAINST':0,'FAVOR':1,'support':1,'neutral':2,'opposed':0,'1':1,'0':0,'2':2,'none':2,'against':0,'favor':1,'negative':0,'favorable':1}
with open('/home/dingdaijun/data_list/dingdaijun/0-dataset/cot-datasets/vast/direct/zero_test.csv') as f:
    ls = csv.DictReader(f)
    for l1 in ls:
        preds1.append(mtm[l1['pred']])
        labels2.append(mtm[l1['stance']])
cal_f1(preds1,labels2)

{'macro_f1_all': 65.11,
 'macro_f': 60.3,
 'f_favor': 63.79,
 'f_against': 56.8,
 'f_none': 74.73}

In [25]:
with open('../datasets/sem16/vast_10/test.csv') as f,open('/home/dingdaijun/data_list/dingdaijun/0-dataset/cot-datasets/vast/direct/zero_test.csv') as f2:
    ls = csv.DictReader(f)
    ls2 = csv.DictReader(f2)
    for index,l1,l2 in zip(range(len(preds)),ls,ls2):
        if preds[index] == labels[index] and preds1[index] != labels2[index]:
            print(l1['Tweet'])
            print(l2['tweet'])
            print(preds[index])
            print(preds1[index])
            print(labels2[index])

"Yet that same Bible gives us Christians a particular anthropology, and reveals to us the meaning of sex and marriage within the cosmic order." And herein lies the problem. If you look to a "holy" book written long, long ago and betraying its human origins on every page for guidance on issues of cosmology, biology, sexuality and other human and natural sciences, you will inevitably be in constant conflict with social progress and, indeed, reality. There is no secular opposition to homosexuality.
yet bible gives us christians particular anthropology reveals us meaning sex marriage within cosmic order. herein lies problem. look holy book written long long ago betraying human origins every page guidance issues cosmology biology sexuality human natural sciences inevitably constant conflict social progress indeed reality. secular opposition homosexuality
1
0
1
Tenure does not mean a teacher cannot lose their job. It requires due process before termination. Before tenure is achieved, a teach